In [3]:
%pip install oracledb

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
import os
sys.path.append(os.path.abspath('../ETL_Scripts'))

In [5]:
import pandas as pd
from functions import insert_data_frame
from database_connection import db_connect

In [6]:
walmart_cursor = db_connect("walmart","walmart","freepdb1")


In [7]:
Categories_df = pd.read_csv("Categories.csv")
Categories_df

,CategoryID,CategoryName,Description
0,1,Electronics,Devices and gadgets
1,2,Clothing,Apparel and accessories
2,3,Books,Literature and reading materials
3,4,Groceries,Daily necessities and food items
4,5,Toys,Children's play items
5,6,Furniture,Home and office furnishings
6,7,Sports,Sports gear and equipment


In [9]:
insert_data_frame(walmart_cursor,"insert into Categories values( :1, :2, :3)",Categories_df)

In [26]:
customer_df = pd.read_csv("Customers.csv")
customer_df.head()

,CustomerID,FirstName,LastName,Email,Phone,Address,MembershipDate
0,1,Brian,Stewart,brian.stewart@gmail.com,(204)898-2161x5216,"19060 Brown Green, South Sandra, CA 39841",2020-03-08
1,2,Kelsey,Medina,kelsey.medina@hotmail.com,300.822.3867x7837,"USCGC Bush, FPO AE 64479",2017-08-14
2,3,Joseph,Hunter,joseph.hunter@hotmail.com,7599008686,"7077 Martinez Junction Apt. 897, South Erica, ...",2023-09-13
3,4,Kathleen,Tucker,kathleen.tucker@hotmail.com,001-236-553-4141x0306,"46334 Michael Plains Apt. 836, Lake Amyland, N...",2021-12-05
4,5,Jacqueline,Cox,jacqueline.cox@hotmail.com,+1-616-869-8535x828,"Unit 9273 Box 8105, DPO AE 72355",2024-11-16


In [27]:
customer_df['MembershipDate'] = pd.to_datetime(customer_df['MembershipDate'])
customer_df['Phone'] = customer_df['Phone'].apply(lambda x: x[0:10])
#customer_df = customer_df.drop(columns=['CustomerID'])
customer_df.head()

,CustomerID,FirstName,LastName,Email,Phone,Address,MembershipDate
0,1,Brian,Stewart,brian.stewart@gmail.com,(204)898-2,"19060 Brown Green, South Sandra, CA 39841",2020-03-08
1,2,Kelsey,Medina,kelsey.medina@hotmail.com,300.822.38,"USCGC Bush, FPO AE 64479",2017-08-14
2,3,Joseph,Hunter,joseph.hunter@hotmail.com,7599008686,"7077 Martinez Junction Apt. 897, South Erica, ...",2023-09-13
3,4,Kathleen,Tucker,kathleen.tucker@hotmail.com,001-236-55,"46334 Michael Plains Apt. 836, Lake Amyland, N...",2021-12-05
4,5,Jacqueline,Cox,jacqueline.cox@hotmail.com,+1-616-869,"Unit 9273 Box 8105, DPO AE 72355",2024-11-16


In [28]:
insert_data_frame(walmart_cursor,"insert into Customers values( :1, :2, :3, :4, :5, :6,:7 )",customer_df)

In [33]:
suppliers_df = pd.read_csv("Operational_data/Suppliers.csv")
suppliers_df.drop(columns=['ProductCategory'],inplace = True)
suppliers_df

,SupplierID,SupplierName,ContactName,Phone,Email,Address
0,1,Tech Supplies Co.,Alice Johnson,123-456-7890,alice@techsupplies.com,"123 Tech Street, Silicon Valley, CA"
1,2,Fashion Trendz,Bob Smith,987-654-3210,bob@fashiontrendz.com,"456 Fashion Blvd, Los Angeles, CA"
2,3,Book Haven,Carla Green,555-666-7777,carla@bookhaven.com,"789 Read Rd, Chicago, IL"
3,4,Fresh Grocer,David Lee,666-777-8888,david@freshgrocer.com,"101 Fresh St, New York, NY"
4,5,Toy Land,Eve White,444-555-6666,eve@toyland.com,"202 Toy St, Dallas, TX"
5,6,Home Comforts,Frank Black,333-444-5555,frank@homecomforts.com,"303 Comfort Rd, Miami, FL"
6,7,SportsGear World,Grace Blue,222-333-4444,grace@sportsgear.com,"404 Sports Ave, Denver, CO"


In [34]:
insert_data_frame(walmart_cursor,"insert into suppliers values( :1, :2, :3, :4, :5, :6)",suppliers_df)

In [36]:
products_df = pd.read_csv("Operational_data/Products.csv")
products_df.head()

,ProductID,ProductName,CategoryID,SupplierID,Price,StockQuantity,ReorderLevel
0,1,Smartphone,1,5,207.58,437,90
1,2,Laptop,1,3,71.19,901,46
2,3,Headphones,1,1,24.91,793,26
3,4,Smartwatch,1,1,27.46,649,14
4,5,Tablet,1,1,377.78,305,90


In [37]:
insert_data_frame(walmart_cursor,"insert into Products values( :1, :2, :3, :4, :5, :6,:7 )",products_df)

In [39]:
orders_df = pd.read_csv("Operational_data/Orders.csv")
orders_df.head()

,OrderID,CustomerID,OrderDate,TotalAmount,PaymentMethod
0,1,1,2024-05-13 13:27:45,55.71,Credit Card
1,2,3,2024-06-04 13:27:45,36.27,PayPal
2,3,67,2024-07-16 13:27:45,448.19,Bank Transfer
3,4,1,2024-08-12 13:27:45,121.06,Credit Card
4,5,72,2025-01-11 13:27:45,435.28,PayPal


In [42]:
orders_df.OrderDate = pd.to_datetime(orders_df['OrderDate'])

In [43]:
insert_data_frame(walmart_cursor,"insert into Orders values( :1, :2, :3, :4, :5 )",orders_df)

In [48]:
orders_details_df = pd.read_csv("Operational_data/order_details.csv")
orders_details_df.head()

,OrderDetailID,OrderID,ProductID,Quantity,UnitPrice,Discount
0,1,63,3,5,137.82,0.16
1,2,70,21,9,165.67,0.21
2,3,70,8,8,183.62,0.22
3,4,76,17,4,198.43,0.19
4,5,3,34,4,53.40,0.07


In [50]:
insert_data_frame(walmart_cursor,"insert into ORDERDETAILS values( :1, :2, :3, :4, :5,:6 )",orders_details_df)